In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import sys
import os
print(os.getcwd())
sys.path.append("../")

from src.data_loader import get_combined_dataloaders
# from src.models import *

import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
%load_ext autoreload
%autoreload 2

/mmfs1/gscratch/rao/aresf/Code/HyperMask/notebooks


/gscratch/rao/aresf/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: '/mmfs1/gscratch/rao/aresf/miniconda3/envs/torch/lib/python3.11/site-packages/torchvision/image.so: undefined symbol: _ZN5torch3jit17parseSchemaOrNameERKNSt7__cxx1112basic_stringIcSt11char_traitsIcESaIcEEE'If you don't plan on using image functionality from `torchvision.io`, you can ignore this warning. Otherwise, there might be something wrong with your environment. Did you have `libjpeg` or `libpng` installed before building `torchvision` from source?
  warn(


In [2]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [35]:
dataset_names = ["mnist", "fashion_mnist"]
train_loader, test_loader = get_combined_dataloaders(
    dataset_names, batch_size=128, equal_samples=True
)

In [44]:
from torch.utils.data import DataLoader, ConcatDataset

In [46]:
d = ConcatDataset(["mnist", "fashion_mnist"])
l = DataLoader(d, batch_size=32)

In [52]:
from torchvision import transforms
from torchvision.datasets import CIFAR10, MNIST

cifar = CIFAR10(
            root="../data",
            train=True,
            download=True,
            transform=transforms.Compose(
            [
                transforms.Resize((28, 28)),
                transforms.Grayscale(),  # Convert to grayscale
                transforms.ToTensor(),
            ]
        )
        )

mnist = MNIST(
            root="../data",
            train=True,
            download=True,
            transform=transforms.Compose(
            [
                transforms.Resize((28, 28)),
                transforms.ToTensor(),
            ]
        )
        )

Files already downloaded and verified


In [54]:
ds = ConcatDataset([cifar, mnist])

In [55]:
l = DataLoader(ds, batch_size=32)

In [58]:
x, y = next(iter(l))

[Dataset CIFAR10
     Number of datapoints: 50000
     Root location: ../data
     Split: Train
     StandardTransform
 Transform: Compose(
                Resize(size=(28, 28), interpolation=bilinear, max_size=None, antialias=warn)
                Grayscale(num_output_channels=1)
                ToTensor()
            ),
 Dataset MNIST
     Number of datapoints: 60000
     Root location: ../data
     Split: Train
     StandardTransform
 Transform: Compose(
                Resize(size=(28, 28), interpolation=bilinear, max_size=None, antialias=warn)
                ToTensor()
            )]

In [36]:
x, y = next(iter(train_loader))

In [6]:
from torch import vmap

class HyperNetwork(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(HyperNetwork, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc = nn.Linear(
            64 * 7 * 7, out_channels * in_channels * kernel_size * kernel_size
        )

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(x.size(0), -1)
        mask = torch.sigmoid(self.fc(x))
        return mask.view(x.size(0), -1)


class PrimaryNetwork(nn.Module):
    def __init__(self, in_channels, out_channels, kernel_size):
        super(PrimaryNetwork, self).__init__()
        self.conv = nn.Conv2d(in_channels, out_channels, kernel_size)

    def forward(self, x, mask):
        masked_weight = self.conv.weight * mask.view_as(self.conv.weight)
        return F.conv2d(x, masked_weight, self.conv.bias)


class GatedConvNet(nn.Module):
    def __init__(self, in_channels=1, out_channels=6, kernel_size=5):
        super(GatedConvNet, self).__init__()
        self.hyper = HyperNetwork(in_channels, out_channels, kernel_size)
        self.primary = PrimaryNetwork(in_channels, out_channels, kernel_size)

    def apply_primary(self, sample, sample_mask):
        return self.primary(
            sample.unsqueeze(0), sample_mask.view_as(self.primary.conv.weight)
        )

    def forward(self, x):
        masks = self.hyper(x)
        return vmap(self.apply_primary)(x, masks).squeeze(1)


In [7]:
model = GatedConvNet()
output = model(x)
print("Output shape:", output.shape)

Output shape: torch.Size([128, 6, 24, 24])


In [40]:
Ys = []
for x, y in train_loader:
    Ys.append(y.numpy())

In [41]:
yu = np.unique(np.hstack(Ys))

In [42]:
yu

array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])

In [7]:
masks = model.hypernetwork(x)

In [11]:
masks[0].shape

torch.Size([128, 150])

In [13]:
model.backbone.conv1.weight.shape

torch.Size([6, 1, 5, 5])

In [15]:
from functorch import vmap

In [ ]:
vmap

In [10]:
model = GatedLeNet()
input_tensor = torch.randn(32, 1, 28, 28)  # Batch of 32 grayscale images of size 28x28
output = model(input_tensor)

TypeError: super(type, obj): obj must be an instance or subtype of type

In [11]:
mask_conv1, mask_conv2, mask_fc1, mask_fc2, mask_fc3 = model.hypernetwork(x)

In [12]:
[x.shape for x in masks]

[torch.Size([128, 150]),
 torch.Size([128, 2400]),
 torch.Size([128, 48000]),
 torch.Size([128, 10080]),
 torch.Size([128, 840])]

In [14]:
mask_conv1.shape

torch.Size([128, 150])